In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.externals.six import StringIO
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error

%matplotlib inline
import sklearn.datasets
from sklearn.datasets import load_iris
from sklearn import tree

df = pd.read_csv('../../datasets/Hitters.csv', index_col = False)

df['League']        = df.League.factorize()[0]
df['Division']      = df.Division.factorize()[0]
df['NewLeague']     = df.NewLeague.factorize()[0]

df.dropna(subset=['Salary'], inplace = True)
df.drop(['Player'], axis = 1, inplace = True)


X = df.drop(['Salary'], axis = 1)
y = df.Salary

df.columns

Index(['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat',
       'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'League', 'Division',
       'PutOuts', 'Assists', 'Errors', 'Salary', 'NewLeague'],
      dtype='object')

# Preprocess

* Split test train
        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=88)


In [32]:
from sklearn.preprocessing import StandardScaler

scal = StandardScaler()
X = scal.fit_transform(X)
y = scal.fit_transform(y)



/Users/aperrier/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/data.py:583: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/Users/aperrier/anaconda/envs/py35/lib/python3.5/site-packages/sklearn/preprocessing/data.py:646: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)


In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=88)

# 1 Basic tree


In [36]:
# ----------------------------------------------------------------------------
#  Basic Decision tree - Baseline
# ----------------------------------------------------------------------------

np.random.seed(88)
clf = DecisionTreeRegressor()
clf.fit(X_train, y_train)

mse = mean_squared_error(y_test, clf.predict(X_test))
print("MSE: %.4f" % mse)


scores = cross_val_score(clf, X, y, cv=10, scoring='mean_squared_error')
print("Default Decision Tree scores: %0.2f (+/- %0.2f) "% (np.mean(scores),np.std(scores) ))


MSE: 0.4518
Default Decision Tree scores: -0.73 (+/- 0.33) 


# Limit the tree

clf = DecisionTreeRegressor(max_depth = 3, min_samples_split = 10)

What MSE do you get?

In [37]:
clf = DecisionTreeRegressor(max_depth = 3, min_samples_split = 10)

clf.fit(X_train, y_train)

mse = mean_squared_error(y_test, clf.predict(X_test))
print("MSE: %.4f" % mse)


scores = cross_val_score(clf, X, y, cv=10, scoring='mean_squared_error')
print("Default Decision Tree scores: %0.2f (+/- %0.2f) "% (np.mean(scores),np.std(scores) ))



MSE: 0.2753
Default Decision Tree scores: -0.54 (+/- 0.27) 


# Bagging 

On default tree

from sklearn.ensemble import BaggingClassifier

        clf = DecisionTreeRegressor()
        bagging = BaggingRegressor(base_estimator = clf, n_estimators = 20, max_samples=0.5, bootstrap=True, oob_score = True)

MSE?


In [39]:
clf = DecisionTreeRegressor()
bagging = BaggingRegressor(base_estimator = clf, n_estimators = 20, max_samples=0.5, bootstrap=True, oob_score = True)
bagging.fit(X,y)

mse = mean_squared_error(y_test, bagging.predict(X_test))
print("MSE: %.4f" % mse)


scores = cross_val_score(clf, X, y, cv=10, scoring='mean_squared_error')
print("Default Decision Tree scores: %0.2f (+/- %0.2f) "% (np.mean(scores),np.std(scores) ))



MSE: 0.1133
Default Decision Tree scores: -0.69 (+/- 0.28) 


# Bagging on constrained tree

use the DecisionTreeRegressor(max_depth = 3, min_samples_split = 10) as the base regressor

In [40]:
clf = DecisionTreeRegressor(max_depth = 3, min_samples_split = 10)
bagging = BaggingRegressor(base_estimator = clf, n_estimators = 20, max_samples=0.5, bootstrap=True, oob_score = True)
bagging.fit(X,y)

mse = mean_squared_error(y_test, bagging.predict(X_test))
print("MSE: %.4f" % mse)


scores = cross_val_score(clf, X, y, cv=10, scoring='mean_squared_error')
print("Default Decision Tree scores: %0.2f (+/- %0.2f) "% (np.mean(scores),np.std(scores) ))


MSE: 0.1592
Default Decision Tree scores: -0.54 (+/- 0.27) 
